In [5]:
from collections import namedtuple
from collections import defaultdict
import pandas as pd
import csv
from heapq import heappop, heappush

# Create a class for the graph

In [59]:
Edge = namedtuple('Edge', 'src dst dist')


class Graph:
    
    def __init__(self):
        self.neighbors = defaultdict(list)
        self.nodes = set()
        
    def add_edge(self, edge: Edge):
        self.neighbors[edge.src].append(edge)
        self.nodes.add(edge.src)
        self.nodes.add(edge.dst)
        
    def __getitem__(self, item):
        return self.neighbors.get(item)

Read csv files

In [7]:
pagelinks = pd.read_csv('simple_english_wiki/simple_english_wiki_pagelinks.csv')
pages = pd.read_csv('simple_english_wiki/simple_english_wiki_pages.csv')

In [44]:
pagelinks

,pl_from,pl_title,pl_to
0,1,1320,39168
1,7560,1320,39168
2,7873,1320,39168
3,8393,1320,39168
4,8719,1320,39168
...,...,...,...
6724923,836310,Anglo-Manipuri_War,836099
6724924,836311,Anglo-Manipuri_War,836099
6724925,836313,Battle_of_Imphal,836107
6724926,836316,Alfredo_Borrero,834696


In [8]:
df = pd.merge(pagelinks, pages, how='inner', left_on='pl_from', right_on='page_id')
df

,pl_from,pl_title,pl_to,page_id,page_title
0,1,1320,39168,1,April
1,1,1502,8495,1,April
2,1,1509,11676,1,April
3,1,1519,10141,1,April
4,1,1533,8497,1,April
...,...,...,...,...,...
6724923,836104,Ethel_Grimwood,836103,836104,Ethel_St._Clair_Grimwood
6724924,836238,Competitive_eating,298847,836238,Eating_contest
6724925,836251,Rio_Tuquesa_tree_frog,831852,836251,Dendropsophus_subocularis
6724926,836311,Anglo-Manipuri_War,836099,836311,Anglo-Manipur_War


Write the update dataframe to the new file

In [9]:
df.to_csv('simple_english_wiki/additional.csv') 

Construct a graph from the given file

In [3]:
def work_with_file(path, g):
    with open(path) as file:
        csvreader = csv.reader(file)
        header = next(csvreader)
        rows = list()
        for row in csvreader:
            edge = Edge(row[1], row[3], 1)
            g.add_edge(edge)

In [6]:
g = Graph()
work_with_file('simple_english_wiki/additional.csv', g)

# Dijkstra's algorithm 

In [16]:
# V - vertices
# E - edges

def dijkstra(graph: Graph, start):
    heap, seen_vertices, min_distances = [(0, start)], set(), {start: 0}
    steps = []
    while heap: # O(V)
        curr_distance, current_vertex = heappop(heap) #O(logV) # start in vertex with min score
        if current_vertex not in seen_vertices:
            
            for _, next_vertex, distance in graph.neighbors.get(current_vertex, []): # O(E)
                if next_vertex in seen_vertices:
                    continue
                    
                prev_min_distance = min_distances.get(next_vertex)
                new_distance = curr_distance + distance
                
                if prev_min_distance is None or new_distance < prev_min_distance:
                    min_distances[next_vertex] = new_distance
                    heappush(heap, (new_distance, next_vertex)) # O(logV)
                    
            seen_vertices.add(current_vertex)

    return min_distances

In [65]:
print("Algorithm id =", str(df[df['page_title']=='Algorithm'].page_id.unique()))
print("Analytics id =", str(df[df['page_title']=='Analytics'].page_id.unique()))

Algorithm id = [170677]
Analytics id = [747593]


Apply dijkstra's algorithm to the page Analytics

In [17]:
min_dist = dijkstra(g, '747593')

Find shortest path to the page Algorithm

In [20]:
min_dist['170677']

4

# Implementation of Dijkstra's algorithm with path

In [21]:
def dijkstra_with_path(graph: Graph, start):
    heap, seen_vertices, min_distances = [(0, start)], set(), {start: (0, [])}
    steps = []
    while heap: # O(V)
        curr_distance, current_vertex = heappop(heap) #O(logV) # start in vertex with min score
        if current_vertex not in seen_vertices:
            
            for _, next_vertex, distance in graph.neighbors.get(current_vertex, []): # O(E)
                if next_vertex in seen_vertices:
                    continue
                    
                prev_min_distance = min_distances.get(next_vertex)
                new_distance = curr_distance + distance
                
                if prev_min_distance is None or new_distance < prev_min_distance[0]:
                    min_distances[next_vertex] = new_distance, min_distances[current_vertex][1] + [current_vertex]
                    heappush(heap, (new_distance, next_vertex)) # O(logV)
                    
            seen_vertices.add(current_vertex)

    return min_distances

Apply dijkstra's algorithm to the page Analytics and check the shortest path to the page Algorithm

In [22]:
min_distances_with_path = dijkstra_with_path(g, '747593')

In [28]:
min_distances_with_path['170677']

(4, ['747593', '31531', '139324', '110'])

Get the title of last page on the path from the page Analytics to the page Algorithm

In [42]:
str(df[df.pl_from == 110]['page_title'].unique())

"['Computer_science']"

In [50]:
def create_graph_with_distances(path, g):
    with open(path) as file:
        csvreader = csv.reader(file)
        header = next(csvreader)
        rows = list()
        for row in csvreader:
            edge = Edge(row[1], row[3], len(row[5]))
            g.add_edge(edge)

In [51]:
graph_with_distances = Graph()
create_graph_with_distances('simple_english_wiki/additional.csv', graph_with_distances)

In [54]:
min_dist = dijkstra(graph_with_distances, '747593')

In [55]:
min_dist['170677']

29

In [56]:
min_distances_with_path = dijkstra_with_path(graph_with_distances, '747593')

In [57]:
min_distances_with_path['170677']

(29, ['747593', '2958', '911', '6309', '5723', '4069'])

In [58]:
str(df[df.pl_from == 4069]['page_title'].unique())

"['Logic']"